In [6]:
import * as React from "react";
import { useState, useEffect, useCallback, useRef } from "react";
import {
  Button,
  Text,
  Spinner,
  makeStyles,
  shorthands,
  tokens,
} from "@fluentui/react-components";
import * as FluentIcons from "@fluentui/react-icons";

const useStyles = makeStyles({
  container: {
    display: "flex",
    flexDirection: "column",
    width: "100%",
    height: "100vh",
    backgroundColor: tokens.colorNeutralBackground3,
    ...shorthands.padding("24px"),
    boxSizing: "border-box",
    overflow: "hidden",
  },
  actionBar: {
    display: "flex",
    flexDirection: "column",
    width: "100%",
    backgroundColor: tokens.colorNeutralBackground2,
    borderRadius: tokens.borderRadiusMedium,
    transition: "all 0.2s ease",
    boxShadow: tokens.shadow4,
    cursor: "pointer",
    ":hover": {
      backgroundColor: tokens.colorNeutralBackground2Hover,
      transform: "translateY(-2px)",
      boxShadow: tokens.shadow8,
    },
  },
  actionContent: {
    display: "flex",
    flexDirection: "column",
    gap: "8px",
    padding: "16px",
  },
  actionTitle: {
    fontSize: tokens.fontSizeBase500,
    fontWeight: tokens.fontWeightSemibold,
    color: tokens.colorNeutralForeground1,
  },
  actionDescription: {
    fontSize: tokens.fontSizeBase300,
    color: tokens.colorNeutralForeground3,
  },
  iconWrapper: {
    display: "flex",
    alignItems: "center",
    justifyContent: "center",
    width: "32px",
    height: "32px",
    borderRadius: "50%",
    backgroundColor: "#106ebe",
    color: tokens.colorNeutralBackground1,
    marginRight: "12px",
  },
  chatPage: {
    position: "absolute",
    top: 0,
    left: 0,
    right: 0,
    bottom: 0,
    backgroundColor: tokens.colorNeutralBackground1,
    display: "flex",
    flexDirection: "column",
    padding: "24px",
  },
  chatHeader: {
    display: "flex",
    flexDirection: "column",
    alignItems: "center",
    marginBottom: "24px",
  },
  chatHeaderTitle: {
    fontSize: tokens.fontSizeBase600,
    fontWeight: tokens.fontWeightSemibold,
    color: tokens.colorNeutralForeground1,
    marginBottom: "8px",
  },
  chatHeaderDescription: {
    fontSize: tokens.fontSizeBase300,
    color: tokens.colorNeutralForeground3,
    textAlign: "center",
  },
  chatMessages: {
    flexGrow: 1,
    overflowY: "auto",
    display: "flex",
    flexDirection: "column",
    gap: "16px",
    marginBottom: "16px",
    padding: "16px",
    border: `1px solid ${tokens.colorNeutralStroke1}`,
    borderRadius: tokens.borderRadiusMedium,
  },
  chatMessage: {
    padding: "12px",
    borderRadius: tokens.borderRadiusMedium,
    maxWidth: "70%",
  },
  userMessage: {
    alignSelf: "flex-end",
    backgroundColor: tokens.colorBrandBackground,
    color: tokens.colorNeutralForegroundInverted,
  },
  assistantMessage: {
    alignSelf: "flex-start",
    backgroundColor: tokens.colorNeutralBackground2,
    color: tokens.colorNeutralForeground1,
  },
  processingMessage: {
    alignSelf: "center",
    backgroundColor: tokens.colorNeutralBackground3,
    color: tokens.colorNeutralForeground3,
    fontStyle: "italic",
  },
  chatInput: {
    display: "flex",
    gap: "8px",
  },
  chatInputField: {
    flex: 1,
    padding: "8px",
    borderRadius: tokens.borderRadiusMedium,
    border: `1px solid ${tokens.colorNeutralStroke1}`,
    fontSize: tokens.fontSizeBase300,
  },
  statusContainer: {
    display: "flex",
    justifyContent: "center",
    alignItems: "center",
    marginTop: "24px",
    minHeight: "60px",
  },
  statusMessage: {
    padding: "12px 24px",
    borderRadius: tokens.borderRadiusMedium,
    backgroundColor: tokens.colorNeutralBackground1,
    color: tokens.colorNeutralForeground1,
    boxShadow: tokens.shadow4,
    textAlign: "center",
    fontSize: tokens.fontSizeBase300,
    fontWeight: tokens.fontWeightSemibold,
    display: "flex",
    flexDirection: "column",
    alignItems: "center",
    justifyContent: "center",
    maxWidth: "80%",
  },
  successMessage: {
    backgroundColor: tokens.colorSuccessBackground,
    color: tokens.colorSuccessForeground1,
  },
  errorMessage: {
    backgroundColor: tokens.colorErrorBackground,
    color: tokens.colorErrorForeground1,
  },
});

const EmailGenerator = () => {
  const styles = useStyles();
  const [isLoading, setIsLoading] = useState(true);
  const [isProcessing, setIsProcessing] = useState(false);
  const [userConfig, setUserConfig] = useState(null);
  const [statusMessage, setStatusMessage] = useState(null);
  const [error, setError] = useState(null);
  const [chatMessages, setChatMessages] = useState([]);
  const [chatInput, setChatInput] = useState("");
  const [isChatActive, setIsChatActive] = useState(false);
  const [processingSteps, setProcessingSteps] = useState([]);
  const chatMessagesRef = useRef(null);

  const fetchUserId = useCallback(async () => {
    try {
      const response = await fetch("http://localhost:8001/getUserId");
      if (!response.ok) throw new Error(`HTTP error! status: ${response.status}`);
      const data = await response.json();
      return data.userId;
    } catch (e) {
      console.error("Error fetching user ID:", e);
      setError("Failed to load user ID. Please try again.");
      return null;
    }
  }, []);

  const fetchUserConfig = useCallback(async (userId) => {
    setIsLoading(true);
    setError(null);
    try {
      console.log(`Fetching user configuration for user: ${userId}`);
      const response = await fetch(`http://localhost:8001/getUserConfig/${userId}`);
      if (!response.ok) throw new Error(`HTTP error! status: ${response.status}`);
      const data = await response.json();
      console.log("Received user configuration:", data);
      setUserConfig(data);
    } catch (e) {
      console.error("Error fetching user configuration:", e);
      setError(`Failed to load user configuration: ${e.message}`);
    } finally {
      setIsLoading(false);
    }
  }, []);

  useEffect(() => {
    const initializeComponent = async () => {
      const userId = await fetchUserId();
      if (userId) {
        await fetchUserConfig(userId);
      }
    };
    initializeComponent();
  }, [fetchUserId, fetchUserConfig]);

  useEffect(() => {
    if (chatMessagesRef.current) {
      chatMessagesRef.current.scrollTop = chatMessagesRef.current.scrollHeight;
    }
  }, [chatMessages]);

  const getEmailContent = async () => {
    return new Promise((resolve, reject) => {
      Office.context.mailbox.item.body.getAsync(Office.CoercionType.Text, (result) => {
        if (result.status === Office.AsyncResultStatus.Succeeded) {
          resolve(result.value);
        } else {
          reject(new Error(result.error.message));
        }
      });
    });
  };

  const wrapInHtml = (content) => {
    // ... (keep the existing wrapInHtml function)
  };

  const handleActionBarClick = () => {
    setIsChatActive(true);
  };

  const handleAction = async (userInput) => {
    setIsProcessing(true);
    setError(null);
    setStatusMessage(null);
    setProcessingSteps([]);
    try {
      const action = userConfig.buttons[0];
      console.log(`Handling action: ${action.label}, User input: ${userInput}`);
      const content = await getEmailContent();
      const payload = {
        userId: userConfig.userId,
        emailContent: content,
        prompt: userInput,
      };

      console.log(`Sending request to: ${action.apiEndpoint}`);
      const response = await fetch(action.apiEndpoint, {
        method: "POST",
        headers: { "Content-Type": "application/json" },
        body: JSON.stringify(payload),
      });

      if (!response.ok) throw new Error(`HTTP error! status: ${response.status}`);
      
      const reader = response.body.getReader();
      const decoder = new TextDecoder();

      while (true) {
        const { value, done } = await reader.read();
        if (done) break;
        const decodedChunk = decoder.decode(value, { stream: true });
        const steps = decodedChunk.split('\n').filter(step => step.trim() !== '');
        setProcessingSteps(prevSteps => [...prevSteps, ...steps]);
      }

      const responseData = await response.json();
      console.log("Received response:", responseData);
      const wrappedContent = wrapInHtml(responseData.body || responseData.generatedContent);
      setChatMessages(prevMessages => [
        ...prevMessages,
        { role: "assistant", content: wrappedContent }
      ]);
    } catch (e) {
      console.error(`Error in handleAction: ${e.message}`);
      setError(`Failed to process your request. Please try again.`);
    } finally {
      setIsProcessing(false);
      setProcessingSteps([]);
    }
  };

  const handleChatSubmit = (e) => {
    e.preventDefault();
    if (chatInput.trim()) {
      setChatMessages(prevMessages => [...prevMessages, { role: "user", content: chatInput }]);
      handleAction(chatInput);
      setChatInput("");
    }
  };

  if (isLoading) {
    return (
      <div className={styles.container}>
        <div className={styles.statusContainer}>
          <Spinner size="large" label="Loading user data..." />
        </div>
      </div>
    );
  }

  if (error) {
    return (
      <div className={styles.container}>
        <div className={styles.statusContainer}>
          <div className={`${styles.statusMessage} ${styles.errorMessage}`}>
            {error}
          </div>
        </div>
      </div>
    );
  }

  if (isChatActive) {
    return (
      <div className={styles.chatPage}>
        <div className={styles.chatHeader}>
          <div className={styles.iconWrapper}>
            <FluentIcons.MailTemplate24Regular />
          </div>
          <Text className={styles.chatHeaderTitle}>{userConfig.buttons[0].label}</Text>
          <Text className={styles.chatHeaderDescription}>{userConfig.buttons[0].description}</Text>
        </div>
        <div className={styles.chatMessages} ref={chatMessagesRef}>
          {chatMessages.map((message, index) => (
            <div
              key={index}
              className={`${styles.chatMessage} ${
                message.role === "user" ? styles.userMessage : styles.assistantMessage
              }`}
            >
              {message.role === "user" ? (
                message.content
              ) : (
                <div dangerouslySetInnerHTML={{ __html: message.content }} />
              )}
            </div>
          ))}
          {isProcessing && (
            <div className={`${styles.chatMessage} ${styles.processingMessage}`}>
              Processing...
              {processingSteps.map((step, index) => (
                <div key={index}>{step}</div>
              ))}
            </div>
          )}
        </div>
        <form onSubmit={handleChatSubmit} className={styles.chatInput}>
          <input
            type="text"
            value={chatInput}
            onChange={(e) => setChatInput(e.target.value)}
            placeholder="Type your message..."
            className={styles.chatInputField}
          />
          <Button type="submit" disabled={isProcessing || !chatInput.trim()}>
            Send
          </Button>
        </form>
      </div>
    );
  }

  return (
    <div className={styles.container}>
      <div className={styles.actionBar} onClick={handleActionBarClick}>
        <div className={styles.actionContent}>
          <div style={{ display: "flex", alignItems: "center" }}>
            <div className={styles.iconWrapper}>
              <FluentIcons.MailTemplate24Regular />
            </div>
            <Text className={styles.actionTitle}>{userConfig.buttons[0].label}</Text>
          </div>
          <Text className={styles.actionDescription}>{userConfig.buttons[0].description}</Text>
        </div>
      </div>
    </div>
  );
};

export default EmailGenerator;

ModuleNotFoundError: No module named 'streamlit'